In [1]:
# ================================================================================
# FEATURE ENGINEERING 
# ================================================================================

import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

print("\n" + "="*80)
print(" FEATURE ENGINEERING")
print("="*80)

# ================================================================================
# 1. ЗАГРУЗКА ДАННЫХ
# ================================================================================

print("\n[1/6] Загрузка данных из preprocessing...")

# названия файлов
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')
complete_df = pd.read_csv('complete_dataset.csv')

with open('holdout_dict.pkl', 'rb') as f:
    holdout_dict = pickle.load(f)

#  структура embeddings
with open('book_embeddings.pkl', 'rb') as f:
    book_embeddings_dict = pickle.load(f)
    # Используем комбинированные эмбеддинги (768-dim)
    book_embeddings = book_embeddings_dict['book_combined']

with open('user_embeddings.pkl', 'rb') as f:
    user_embeddings_dict = pickle.load(f)
    user_embeddings_train = user_embeddings_dict['train']
    user_embeddings_test = user_embeddings_dict['test']

# Создаем book_tags_clean из complete_dataset
print(" Создание book_tags_clean...")
book_tags_clean = {}
for _, row in complete_df[['book_id', 'tags_text']].drop_duplicates('book_id').iterrows():
    if pd.notna(row['tags_text']) and row['tags_text'].strip() != '':
        book_tags_clean[row['book_id']] = row['tags_text'].split()
    else:
        book_tags_clean[row['book_id']] = []

print(f"  Загружено:")
print(f"     • Train: {len(train_df):,} оценок")
print(f"     • Test: {len(test_df):,} оценок")
print(f"     • Book embeddings: {len(book_embeddings):,} книг × 768")
print(f"     • User embeddings (train): {len(user_embeddings_train):,} × 768")
print(f"     • User embeddings (test): {len(user_embeddings_test):,} × 768")
print(f"     • Book tags: {len(book_tags_clean):,} книг")
print(f"     • Holdout users: {len(holdout_dict):,}")

# ================================================================================
# 2. USER FEATURES (4 признака)
# ================================================================================

print("\n" + "="*80)
print("[2/6] СОЗДАНИЕ USER FEATURES")
print("="*80)

print("\n  Вычисление статистик пользователей...")

# 2.1. Базовая статистика
user_stats = train_df.groupby('user_id').agg({
    'rating': ['mean', 'count'], 'book_id': 'nunique'
}).reset_index()

user_stats.columns = ['user_id', 'avg_user_rating', 'ratings_count', 'unique_books_count']  

print(f"     avg_user_rating, ratings_count, unique_books_count")

# 2.2. Tag vocabulary size (разнообразие тегов в истории)
print("\n  Вычисление tag_vocab_size...")

user_tag_vocab = {}

for user_id, group in tqdm(train_df.groupby('user_id'), desc="     Processing users"):
    user_tag_set = set()
    
    for book_id in group['book_id']:
        if book_id in book_tags_clean:
            user_tag_set.update(book_tags_clean[book_id])
    
    user_tag_vocab[user_id] = len(user_tag_set)

user_stats['tag_vocab_size'] = user_stats['user_id'].map(user_tag_vocab).fillna(0)

print(f"     tag_vocab_size")

# 2.3. Activity score (комбинированная метрика активности)
print("\n  Вычисление activity_score...")

def minmax_scale(series):
    """Min-max нормализация"""
    min_val = series.min()
    max_val = series.max()
    if max_val == min_val:
        return pd.Series(np.ones(len(series)), index=series.index)
    return (series - min_val) / (max_val - min_val)

# Нормализуем ratings_count и tag_vocab_size
norm_ratings = minmax_scale(user_stats['ratings_count'])
norm_vocab = minmax_scale(user_stats['tag_vocab_size'])

# Activity = 0.7 * norm(ratings) + 0.3 * norm(vocab)
user_stats['activity_score'] = 0.7 * norm_ratings + 0.3 * norm_vocab

print(f"      activity_score")

# 2.4. Сегментация пользователей
print("\n  Сегментация пользователей...")

def classify_user_segment(unique_books): 
    """Классифицирует пользователя по активности"""
    if unique_books < 5:
        return 'new'
    elif 5 <= unique_books <= 10:
        return 'inactive'
    elif 11 <= unique_books <= 60:
        return 'active'
    else:
        return 'very_active'

user_stats['segment'] = user_stats['unique_books_count'].apply(classify_user_segment)
# Удаляем unique_books_count (нужен был только для сегментации)
user_stats = user_stats.drop(columns=['unique_books_count'])

# Статистика по сегментам
print(f"\n   Распределение по сегментам:")
segment_counts = user_stats['segment'].value_counts()
for segment, count in segment_counts.items():
    pct = count / len(user_stats) * 100
    print(f"     • {segment:15s}: {count:>6,} ({pct:>5.1f}%)")

# Сохраняем user features
user_stats.to_csv('user_features.csv', index=False)
print(f"\n   User features сохранены: user_features.csv")
print(f"     Признаков: {len(user_stats.columns) - 1} (кроме user_id)")

# ================================================================================
# 3. BOOK FEATURES (3 признака)
# ================================================================================

print("\n" + "="*80)
print("[3/6] СОЗДАНИЕ BOOK FEATURES")
print("="*80)

print("\n  Вычисление статистик книг...")

# 3.1. Базовая статистика
book_stats = train_df.groupby('book_id').agg({
    'rating': ['mean', 'count']
}).reset_index()

book_stats.columns = ['book_id', 'book_avg_rating', 'book_ratings_count']

print(f"      book_avg_rating, book_ratings_count")

# 3.2. Book popularity (комбинированная метрика)
print("\n  Вычисление book_popularity...")

# Нормализуем
norm_book_ratings = minmax_scale(book_stats['book_ratings_count'])
norm_book_avg = minmax_scale(book_stats['book_avg_rating'])

# Popularity = 0.7 * norm(ratings_count) + 0.3 * norm(avg_rating)
book_stats['book_popularity'] = 0.7 * norm_book_ratings + 0.3 * norm_book_avg

print(f"     book_popularity")

# Сохраняем book features
book_stats.to_csv('book_features.csv', index=False)
print(f"\n  Book features сохранены: book_features.csv")
print(f"     Признаков: {len(book_stats.columns) - 1} (кроме book_id)")

# ================================================================================
# 4. INTERACTION FEATURES (6 признаков) 
# ================================================================================

print("\n" + "="*80)
print("[4/6] СОЗДАНИЕ INTERACTION FEATURES")
print("="*80)

print("\n Это займет время (много пар user-book)...")
print(f"  Будет создано признаков для: {len(train_df):,} train + {len(test_df):,} test")

# Размерность эмбеддингов 768
EMB_DIM = 768

# Функция БЕЗ data leakage для train
def create_interaction_features_train(idx, user_id, book_id, train_df, book_tags_clean, 
                                       user_embeddings, book_embeddings):
    """
    Создает 6 признаков взаимодействия для TRAIN
    Исключает текущую книгу из истории пользователя!
    """
    features = {}
    
    # --- Tag-based features (БЕЗ текущей книги в истории) ---
    # Получаем историю юзера ДО текущей строки (по индексу)
    user_history = train_df[(train_df['user_id'] == user_id) & (train_df.index < idx)]
    
    user_tags = set()
    for bid in user_history['book_id']:
        if bid in book_tags_clean:
            user_tags.update(book_tags_clean[bid])
    
    book_tags = set(book_tags_clean.get(book_id, []))
    
    # Пересечение тегов
    intersection = user_tags & book_tags
    union = user_tags | book_tags
    
    features['tag_overlap_count'] = len(intersection)
    features['tag_overlap_ratio'] = len(intersection) / len(book_tags) if book_tags else 0
    features['tag_jaccard'] = len(intersection) / len(union) if union else 0
    features['history_similarity'] = 0.6 * features['tag_overlap_ratio'] + 0.4 * features['tag_jaccard']
    
    # --- Embedding-based features ---
    user_emb = user_embeddings.get(user_id, np.zeros(EMB_DIM))
    book_emb = book_embeddings.get(book_id, np.zeros(EMB_DIM))
    
    # Обработка нулевых векторов
    user_norm = np.linalg.norm(user_emb)
    book_norm = np.linalg.norm(book_emb)
    
    if user_norm == 0 or book_norm == 0:
        features['embedding_cosine_sim'] = 0.0
        features['embedding_euclidean_dist'] = 1.0  # максимальное расстояние
    else:
        # Косинусная близость
        features['embedding_cosine_sim'] = np.dot(user_emb, book_emb) / (user_norm * book_norm)
        
        # ИСПРАВЛЕНИЕ 7: Нормализованное евклидово расстояние
        # Нормализуем векторы
        user_emb_norm = user_emb / user_norm
        book_emb_norm = book_emb / book_norm
        features['embedding_euclidean_dist'] = np.linalg.norm(user_emb_norm - book_emb_norm) / np.sqrt(2)
    
    return features

# Функция для test (используем ВСЮ историю train)
def create_interaction_features_test(user_id, book_id, user_tags_dict, book_tags_clean, 
                                      user_embeddings, book_embeddings):
    """
    Создает 6 признаков взаимодействия для TEST
    Использует полную историю пользователя из train
    """
    features = {}
    
    # --- Tag-based features ---
    user_tags = user_tags_dict.get(user_id, set())
    book_tags = set(book_tags_clean.get(book_id, []))
    
    intersection = user_tags & book_tags
    union = user_tags | book_tags
    
    features['tag_overlap_count'] = len(intersection)
    features['tag_overlap_ratio'] = len(intersection) / len(book_tags) if book_tags else 0
    features['tag_jaccard'] = len(intersection) / len(union) if union else 0
    features['history_similarity'] = 0.6 * features['tag_overlap_ratio'] + 0.4 * features['tag_jaccard']
    
    # --- Embedding-based features ---
    user_emb = user_embeddings.get(user_id, np.zeros(EMB_DIM))
    book_emb = book_embeddings.get(book_id, np.zeros(EMB_DIM))
    
    user_norm = np.linalg.norm(user_emb)
    book_norm = np.linalg.norm(book_emb)
    
    if user_norm == 0 or book_norm == 0:
        features['embedding_cosine_sim'] = 0.0
        features['embedding_euclidean_dist'] = 1.0
    else:
        features['embedding_cosine_sim'] = np.dot(user_emb, book_emb) / (user_norm * book_norm)
        
        user_emb_norm = user_emb / user_norm
        book_emb_norm = book_emb / book_norm
        features['embedding_euclidean_dist'] = np.linalg.norm(user_emb_norm - book_emb_norm) / np.sqrt(2)
    
    return features

# Создаем interaction features для TRAIN
print(f"\n  Создание interaction features для train...")

train_interactions = []

for idx, row in tqdm(train_df.iterrows(), total=len(train_df), desc="     Train"):
    features = create_interaction_features_train(
        idx,
        row['user_id'], 
        row['book_id'],
        train_df,
        book_tags_clean,
        user_embeddings_train,
        book_embeddings
    )
    
    # Добавляем основные поля
    features['user_id'] = row['user_id']
    features['book_id'] = row['book_id']
    features['rating'] = row['rating']
    
    train_interactions.append(features)

train_features_df = pd.DataFrame(train_interactions)

print(f"  Train interaction features: {len(train_features_df):,}")

# Создаем словарь user_tags для TEST (полная история train)
print(f"\n  Подготовка user tags для test...")

user_tags_dict = {}

for user_id, group in tqdm(train_df.groupby('user_id'), desc="     User tags"):
    user_tags = set()
    for book_id in group['book_id']:
        if book_id in book_tags_clean:
            user_tags.update(book_tags_clean[book_id])
    user_tags_dict[user_id] = user_tags

print(f"  User tags готовы: {len(user_tags_dict):,} пользователей")

# Создаем interaction features для TEST
print(f"\n  Создание interaction features для test...")

test_interactions = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="     Test"):
    features = create_interaction_features_test(
        row['user_id'], 
        row['book_id'],
        user_tags_dict,
        book_tags_clean,
        user_embeddings_test,  # Используем test embeddings!
        book_embeddings
    )
    
    features['user_id'] = row['user_id']
    features['book_id'] = row['book_id']
    features['rating'] = row['rating']
    
    test_interactions.append(features)

test_features_df = pd.DataFrame(test_interactions)

print(f"  Test interaction features: {len(test_features_df):,}")



 FEATURE ENGINEERING

[1/6] Загрузка данных из preprocessing...
 Создание book_tags_clean...
  Загружено:
     • Train: 874,496 оценок
     • Test: 107,260 оценок
     • Book embeddings: 10,000 книг × 768
     • User embeddings (train): 53,424 × 768
     • User embeddings (test): 35,659 × 768
     • Book tags: 10,000 книг
     • Holdout users: 35,659

[2/6] СОЗДАНИЕ USER FEATURES

  Вычисление статистик пользователей...
     avg_user_rating, ratings_count, unique_books_count

  Вычисление tag_vocab_size...


     Processing users: 100%|██████████| 53424/53424 [00:04<00:00, 11887.91it/s]


     tag_vocab_size

  Вычисление activity_score...
      activity_score

  Сегментация пользователей...

   Распределение по сегментам:
     • new            : 25,571 ( 47.9%)
     • active         : 16,240 ( 30.4%)
     • inactive       :  8,343 ( 15.6%)
     • very_active    :  3,270 (  6.1%)

   User features сохранены: user_features.csv
     Признаков: 5 (кроме user_id)

[3/6] СОЗДАНИЕ BOOK FEATURES

  Вычисление статистик книг...
      book_avg_rating, book_ratings_count

  Вычисление book_popularity...
     book_popularity

  Book features сохранены: book_features.csv
     Признаков: 3 (кроме book_id)

[4/6] СОЗДАНИЕ INTERACTION FEATURES

 Это займет время (много пар user-book)...
  Будет создано признаков для: 874,496 train + 107,260 test

  Создание interaction features для train...


     Train: 100%|██████████| 874496/874496 [27:40<00:00, 526.64it/s]


  Train interaction features: 874,496

  Подготовка user tags для test...


     User tags: 100%|██████████| 53424/53424 [00:06<00:00, 7790.42it/s]


  User tags готовы: 53,424 пользователей

  Создание interaction features для test...


     Test: 100%|██████████| 107260/107260 [00:09<00:00, 10794.18it/s]


  Test interaction features: 107,260


In [2]:

# ================================================================================
# 5. ОБЪЕДИНЕНИЕ ВСЕХ ПРИЗНАКОВ
# ================================================================================

print("\n" + "="*80)
print("[5/6] ОБЪЕДИНЕНИЕ ВСЕХ ПРИЗНАКОВ")
print("="*80)

print("\n  Мердж с user и book features...")

# Train
train_full = train_features_df.merge(user_stats, on='user_id', how='left')
train_full = train_full.merge(book_stats, on='book_id', how='left')

# Test
test_full = test_features_df.merge(user_stats, on='user_id', how='left')
test_full = test_full.merge(book_stats, on='book_id', how='left')

print(f"     Train: {train_full.shape}")
print(f"     Test: {test_full.shape}")

# Добавляем признаки из preprocessing
print("\n  Добавление признаков из preprocessing...")

# Признаки которые нужно добавить
extra_cols = ['language_code_encoded', 'year_normalized', 'publication_era', 'average_rating']

# Для train
train_extra = train_df[['user_id', 'book_id'] + extra_cols].copy()
# Создаем уникальный ключ (для случая дубликатов)
train_extra['row_num'] = train_extra.groupby(['user_id', 'book_id']).cumcount()
train_full['row_num'] = train_full.groupby(['user_id', 'book_id']).cumcount()

train_full = train_full.merge(train_extra, on=['user_id', 'book_id', 'row_num'], how='left')
train_full = train_full.drop(columns=['row_num'])

# Для test
test_extra = test_df[['user_id', 'book_id'] + extra_cols].copy()
test_extra['row_num'] = test_extra.groupby(['user_id', 'book_id']).cumcount()
test_full['row_num'] = test_full.groupby(['user_id', 'book_id']).cumcount()

test_full = test_full.merge(test_extra, on=['user_id', 'book_id', 'row_num'], how='left')
test_full = test_full.drop(columns=['row_num'])

print(f"     Добавлено {len(extra_cols)} признаков из preprocessing")
print(f"      Train final: {train_full.shape}")
print(f"     Test final: {test_full.shape}")



[5/6] ОБЪЕДИНЕНИЕ ВСЕХ ПРИЗНАКОВ

  Мердж с user и book features...
     Train: (874496, 17)
     Test: (107260, 17)

  Добавление признаков из preprocessing...
     Добавлено 4 признаков из preprocessing
      Train final: (874496, 21)
     Test final: (107260, 21)


In [3]:

# Проверка пропусков
print("\n  🔍 Проверка пропусков...")
train_missing = train_full.isnull().sum()
test_missing = test_full.isnull().sum()

if train_missing.sum() > 0:
    print(f"       Train пропуски:")
    print(train_missing[train_missing > 0])
else:
    print(f"      Train: пропусков нет")

if test_missing.sum() > 0:
    print(f"   Test пропуски (cold start books):")
    print(test_missing[test_missing > 0])
    
    print(f"\n      Заполнение пропусков...")
    
    # 1. book_avg_rating: используем average_rating из Goodreads
    mask = test_full['book_avg_rating'].isna()
    if mask.sum() > 0:
        test_full.loc[mask, 'book_avg_rating'] = test_full.loc[mask, 'average_rating']
    
    # 2. Если и Goodreads нет - глобальное среднее
    test_full['book_avg_rating'].fillna(train_full['book_avg_rating'].median(), inplace=True)
    
    # 3. book_ratings_count = минимум (новая книга)
    test_full['book_ratings_count'].fillna(1, inplace=True)
    
    # 4. book_popularity: пересчитываем для холодных книг
    mask = test_full['book_popularity'].isna()
    if mask.sum() > 0:
        # Нормализуем относительно train
        max_ratings = train_full['book_ratings_count'].max()
        min_rating = train_full['book_avg_rating'].min()
        max_rating = train_full['book_avg_rating'].max()
        
        norm_count = (test_full.loc[mask, 'book_ratings_count'] - 1) / max_ratings
        norm_rating = (test_full.loc[mask, 'book_avg_rating'] - min_rating) / (max_rating - min_rating)
        test_full.loc[mask, 'book_popularity'] = 0.7 * norm_count + 0.3 * norm_rating
    
    print(f"      Заполнено {test_missing[test_missing > 0].sum()} пропусков")
    
    # Проверяем что пропусков не осталось
    feature_cols = [
    # Interaction (6)
    'tag_overlap_count', 'tag_overlap_ratio', 'tag_jaccard', 
    'history_similarity', 'embedding_cosine_sim', 'embedding_euclidean_dist',
    # User (4) - БЕЗ unique_books_count!
    'avg_user_rating', 'ratings_count', 'tag_vocab_size', 'activity_score',
    # Book (3)
    'book_avg_rating', 'book_ratings_count', 'book_popularity',
    # Preprocessing (4)
    'language_code_encoded', 'year_normalized', 'publication_era', 'average_rating'
]
    test_missing_after = test_full[feature_cols].isnull().sum()
    if test_missing_after.sum() > 0:
        print(f"       Остались пропуски:")
        print(test_missing_after[test_missing_after > 0])
    else:
        print(f"      Все пропуски заполнены!")
else:
    print(f"  Test: пропусков нет")

# Сохраняем
train_full.to_csv('train_features_full.csv', index=False)
test_full.to_csv('test_features_full.csv', index=False)

print(f"\n   Полные датасеты сохранены:")
print(f"     • train_features_full.csv")
print(f"     • test_features_full.csv")



  🔍 Проверка пропусков...
      Train: пропусков нет
   Test пропуски (cold start books):
book_avg_rating       400
book_ratings_count    400
book_popularity       400
dtype: int64

      Заполнение пропусков...
      Заполнено 1200 пропусков
      Все пропуски заполнены!


C:\Users\admin\AppData\Local\Temp\ipykernel_18488\3582154444.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_full['book_avg_rating'].fillna(train_full['book_avg_rating'].median(), inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_18488\3582154444.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are settin


   Полные датасеты сохранены:
     • train_features_full.csv
     • test_features_full.csv


In [4]:

# ================================================================================
# 6. ФИНАЛЬНАЯ СТАТИСТИКА
# ================================================================================

print("\n" + "="*80)
print("[6/6] ФИНАЛЬНАЯ СТАТИСТИКА")
print("="*80)


print(f"\n Созданные признаки:")

print(f"\n   Interaction Features (6):")
print(f"     • tag_overlap_count - количество общих тегов")
print(f"     • tag_overlap_ratio - доля пересечения")
print(f"     • tag_jaccard - коэффициент Жаккара")
print(f"     • history_similarity - взвешенная схожесть")
print(f"     • embedding_cosine_sim - косинусная близость (768-dim)")
print(f"     • embedding_euclidean_dist - евклидово расстояние (нормализовано)")

print(f"\n   User Features (4):")
print(f"     • avg_user_rating - средний рейтинг пользователя")
print(f"     • ratings_count - активность")
print(f"     • tag_vocab_size - разнообразие интересов")
print(f"     • activity_score - комбинированная метрика")

print(f"\n   Book Features (3):")
print(f"     • book_avg_rating - средний рейтинг книги")
print(f"     • book_ratings_count - известность")
print(f"     • book_popularity - комбинированная популярность")

print(f"\n   From Preprocessing (4):")
print(f"     • language_code_encoded - язык книги (закодирован)")
print(f"     • year_normalized - год публикации (нормализован)")
print(f"     • publication_era - эпоха (0-7)")
print(f"     • average_rating - средний рейтинг из Goodreads")

print(f"\n   Дополнительно:")
print(f"     • segment - категориальный (new/inactive/active/very_active)")

total_features = len(feature_cols)
print(f"\n   Всего числовых признаков: {total_features}")

# Статистика по признакам
print(f"\n Статистика признаков (train):")
print("\n", train_full[feature_cols].describe())

# Сохранение метаданных
metadata = {
    'num_users': len(user_stats),
    'num_books': len(book_stats),
    'train_size': len(train_full),
    'test_size': len(test_full),
    'num_features': total_features,
    'feature_names': feature_cols,
    'interaction_features': feature_cols[:6],
    'user_features': feature_cols[6:10],
    'book_features': feature_cols[10:13],
    'preprocessing_features': feature_cols[13:17],
    'segment_distribution': segment_counts.to_dict(),
    'embedding_dim': EMB_DIM
}

with open('features_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

print(f"\n   Метаданные сохранены: features_metadata.pkl")

print("\n" + "="*80)
print(" FEATURE ENGINEERING ЗАВЕРШЕН!")
print("="*80)

print(f"\n Созданные файлы:")
print(f"  1. user_features.csv")
print(f"  2. book_features.csv")
print(f"  3. train_features_full.csv")
print(f"  4. test_features_full.csv")
print(f"  5. features_metadata.pkl")





[6/6] ФИНАЛЬНАЯ СТАТИСТИКА

 Созданные признаки:

   Interaction Features (6):
     • tag_overlap_count - количество общих тегов
     • tag_overlap_ratio - доля пересечения
     • tag_jaccard - коэффициент Жаккара
     • history_similarity - взвешенная схожесть
     • embedding_cosine_sim - косинусная близость (768-dim)
     • embedding_euclidean_dist - евклидово расстояние (нормализовано)

   User Features (4):
     • avg_user_rating - средний рейтинг пользователя
     • ratings_count - активность
     • tag_vocab_size - разнообразие интересов
     • activity_score - комбинированная метрика

   Book Features (3):
     • book_avg_rating - средний рейтинг книги
     • book_ratings_count - известность
     • book_popularity - комбинированная популярность

   From Preprocessing (4):
     • language_code_encoded - язык книги (закодирован)
     • year_normalized - год публикации (нормализован)
     • publication_era - эпоха (0-7)
     • average_rating - средний рейтинг из Goodreads

   Доп